In [ ]:
import pandas as pd
df_num = pd.read_csv('final_dataset_num.csv')
df_num.head()

,Unnamed: 0,time,place,temp(°C),rain(mm),wind(m/s),NO2(ppm),ozone(ppm),CO2(ppm),SO2(ppm),dust,micro dust,rent_num
0,0,2017-06-05,서초구,23.5,0.0,1.3,0.029,0.047,0.4,0.005,29.0,14.0,25
1,1,2017-06-06,서초구,18.9,8.5,1.8,0.017,0.042,0.3,0.005,21.0,10.0,14
2,2,2017-06-07,서초구,18.2,9.5,1.6,0.020,0.030,0.4,0.005,18.0,9.0,18
3,3,2017-06-08,서초구,20.5,0.0,1.3,0.019,0.056,0.6,0.006,47.0,28.0,34
4,4,2017-06-09,서초구,22.0,0.0,1.7,0.017,0.061,0.5,0.006,60.0,33.0,25


In [16]:
df_num.drop(['time', 'place'], axis = 1, inplace = True)
df_num.head()

,Unnamed: 0,temp(°C),rain(mm),wind(m/s),NO2(ppm),ozone(ppm),CO2(ppm),SO2(ppm),dust,micro dust,rent_num
0,0,23.5,0.0,1.3,0.029,0.047,0.4,0.005,29.0,14.0,25
1,1,18.9,8.5,1.8,0.017,0.042,0.3,0.005,21.0,10.0,14
2,2,18.2,9.5,1.6,0.020,0.030,0.4,0.005,18.0,9.0,18
3,3,20.5,0.0,1.3,0.019,0.056,0.6,0.006,47.0,28.0,34
4,4,22.0,0.0,1.7,0.017,0.061,0.5,0.006,60.0,33.0,25


In [17]:
# preprocessing - 전처리 분리
#target_variable : rent_num
from sklearn.model_selection import train_test_split
X, y = df_num.iloc[:,:-1].values, df_num.iloc[:,-1].values
train_X, test_X, train_y, test_y = train_test_split(X, y , test_size = 0.3, random_state = 0)

In [35]:
#standardization

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_X_std = sc.fit_transform(train_X)
test_X_std = sc.transform(test_X)

In [19]:
import numpy as np
cov_mat = np.cov(train_X_std.T)

In [24]:
#공분산 이용 - 고유값 분해
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)
print('Eigenvalues : %s' %eigen_vals)

Eigenvalues : [3.11965217 0.13449221 0.30677796 0.35797734 1.36382959 0.69610324
 1.11752085 1.03250907 0.90522926 0.96727724]


In [25]:
tot = sum(eigen_vals)
var_exp = [(i / tot) for i in sorted(eigen_vals, reverse = True)]
var_exp

[0.31192251676824395,
 0.1363642918693529,
 0.11173678933140524,
 0.10323677433237137,
 0.09671448428655816,
 0.09051053596259498,
 0.06960079592525145,
 0.035792834514194086,
 0.030673597238006094,
 0.013447379772021852]

In [26]:
#주성분과 주성분 행렬 쌍을 이용하여 투영행렬 생성

eigen_pairs = [(np.abs(eigen_vals[i]), eigen_vecs[:,i]) for i in range(len(eigen_vals))]
eigen_pairs.sort(reverse = True)
eigen_pairs

[(3.1196521663363854,
  array([-0.03579519,  0.30674032,  0.16779879,  0.17950962, -0.09283535,
          0.22745814, -0.47610531, -0.33095322, -0.47005068, -0.4761716 ])),
 (1.3638295912354081,
  array([-0.22048908,  0.45195397, -0.20741606,  0.01205217, -0.05774518,
          0.6921862 , -0.1269732 ,  0.21594324,  0.3023858 ,  0.25943863])),
 (1.117520852642334,
  array([-0.1963733 , -0.48557813, -0.32020853,  0.7494828 , -0.16950971,
          0.07082453, -0.13730737,  0.02000326,  0.08810868, -0.02504394])),
 (1.0325090667656196,
  array([-0.81530747, -0.05826533,  0.2273406 , -0.05736756,  0.46509902,
         -0.1472966 , -0.10135585,  0.15522129, -0.02209152, -0.06352967])),
 (0.9672772377790212,
  array([ 0.30928073,  0.00774778, -0.43124145,  0.07506409,  0.82112672,
          0.10433206, -0.05752745, -0.13922268, -0.04977205, -0.04876178])),
 (0.9052292617969901,
  array([-0.255683  , -0.06515015, -0.73417373, -0.46733786, -0.25628382,
         -0.12407011, -0.06921345,  0.02

In [27]:
w = np.hstack((eigen_pairs[0][1][:, np.newaxis],
               eigen_pairs[1][1][:, np.newaxis]))
print(w)

[[-0.03579519 -0.22048908]
 [ 0.30674032  0.45195397]
 [ 0.16779879 -0.20741606]
 [ 0.17950962  0.01205217]
 [-0.09283535 -0.05774518]
 [ 0.22745814  0.6921862 ]
 [-0.47610531 -0.1269732 ]
 [-0.33095322  0.21594324]
 [-0.47005068  0.3023858 ]
 [-0.4761716   0.25943863]]


In [28]:
train_X_pca = train_X_std.dot(w)

In [30]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes = (50, 30))
mlp.fit(train_X_pca, train_y)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50, 30), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [33]:
mlp.score(train_X_pca, train_y)

0.1308513550506433

In [36]:
test_X_pca = test_X_std.dot(w)
pred = mlp.predict(test_X_pca)
pred

array([ 901,  454, 6748, ...,  796, 8787, 2324])

In [37]:
import pandas as pd
confusion_matrix = pd.crosstab(test_y, pred, margins = True, rownames = ['True'], colnames = ['Pred'])
confusion_matrix

Pred,7,26,28,40,42,43,46,50,54,56,58,63,72,75,76,77,82,83,89,92,93,94,99,101,104,106,112,114,116,123,125,126,129,131,132,133,137,149,153,157,...,6630,6677,6725,6748,6771,6802,6883,6917,6937,6942,6974,6981,7067,7068,7114,7180,7292,7508,7510,7608,7706,7729,7784,8019,8111,8134,8354,8382,8445,8480,8498,8565,8610,8787,8900,8915,8926,9045,9472,All
True,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
18,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9061,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9094,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9481,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [38]:
mlp.score(test_X_pca, test_y)

0.0